## <center> Plotting B-spline curves defined by  `BasicBSpline.jl` </center>

Docs: https://hyrodium.github.io/BasicBSpline.jl/dev/

In [1]:
using BasicBSpline, PlotlyJS

function bsplcurve(p::Int, knot:: KnotVector{Float64}, ctrl:: Vector{Vector{T}}, N::Int) where T
    """
    p: degree of B-spline basis functions
    knot: the sequence of m knots, m-p >p 
    ctrl: the vector of control points; length(ctrl)= n =  m-p-1
    N: the number of points to be computed on the B-spline curve
    returns N points on the B-spline curve defined by the knots and ctrl
    """
    m = length(knot)
    BS = BSplineSpace{p}(knot) # B-spline space
    n  = dim(BS)
    @assert length(ctrl) == n
    I = domain(BS)
    a, b = extrema(I)
    t = LinRange(a, b, N)
    bs_curve= Vector{Float64}[]
    for s in t
        basis= [bsplinebasis(BS, i, s) for i in 1:n]
        push!(bs_curve, sum(ctrl[i,]*basis[i] for i in 1:n))
    end
    return bs_curve
end;    

WebIO._IJuliaInit()

In [2]:
function figbspline2d(ctrl:: Vector{Vector{T}}, bs_pts::Vector{Vector{Float64}}; 
                    width=600, height=600) where T
    # function to display a 2D B-spline curve as a PlotlyJS.jl plot
    c = mapreduce(permutedims, vcat, ctrl)
    bp = mapreduce(permutedims, vcat, bs_pts)
    fig = Plot(scatter(x=c[:, 1], y=c[:, 2],                                          
                       name="ctrl polygon", marker_size=8))
    addtraces!(fig, scatter(x=bp[:, 1], y=bp[:, 2],      
                            name="b-spline", mode="lines", line_color="red"))
    relayout!(fig, width=width, height=height) 
    return fig
end; 

**Example of B-spline curve interpolating the  first and last  control point**

In [4]:
p = 3 #degree of BS polynomials
#end knots must have multiplicity p+1 to ensure that the B-spline curve contains the end control points
knot = KnotVector(1, 1, 1, 1, 1.5, 2.3, 3., 3.5, 3.5, 3.5, 3.5) 
m = length(knot)
#Give m-p-1  control points
ctrl=[[0., 0.45], [-0.5, 0.75], [-1.,2.0], [0.5, 3.2], 
      [1.6, 1.0], [1.3, -0.3], [0.75, -1.0]];

bs_pts = bsplcurve(p, knot, ctrl, 200) 
fig1 = figbspline2d(ctrl, bs_pts; height=450)
display(fig1)

data: [
  "scatter with fields marker, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, and width"

If a B-spline curve interpolates the end control points ctrl[1], ctrl[n], then the tangents to 
ctrl[1], respectively ctrl[n], have the directions of the  lines through (ctrl[1], ctrl[2]), respectively (ctrl[n-1], ctrl[n]).

Let us generate the B-spline curve that interpolates the first and last control point and moreover it has an interior knot, multiple of  order p=3. If it is the knot $knot[j+1]=knot[j+2]+knot[j+3]$, for some  $j\in \{p+2, 1, \ldots, m-p-1\}$, then the B-spline curve defined by  such a knot and a control polygon, passes through the control point ctrl[j].

In [5]:
p=3
knot = (p+1)*KnotVector(0) + KnotVector(3) + p*KnotVector(5) + KnotVector(8., 10)+(p+1)*KnotVector(13)

KnotVector([0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 5.0, 5.0, 8.0, 10.0, 13.0, 13.0, 13.0, 13.0])

In [6]:
m = length(knot)

14

Hence we need n =14-4=10 control points:

In [7]:
ctrl= [[0., 0.], [-1.0, 2.0], [0.75, 4.0], [2.3, 5.], [4.0, 3.8],
       [2, 2.75], [5.0, -1.0], [6.25, 0.2], [7.14, 3.5], [6.3, 4.45]]
bs_pts = bsplcurve(p, knot, ctrl, 200)   
fig2 = figbspline2d(ctrl, bs_pts; height=450)
display(fig2)

data: [
  "scatter with fields marker, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, and width"

Since  we have knot[6]=knot[7]=knot[8], the B-spline curve contains the control point ctrl[5], and the control polygon segments, incident in ctrl[5], are tangent to the left/right arc through ctrl[5].

**B-spline-curve that does not interpolate the first and the last control point**

In [8]:
p=3
knot = KnotVector(0, 1, 2, 3, 4, 5, 6, 7, 8) #m=9
ctrl = [[0, 0.0], [-1.0, 2.3], [0.25, 3], [1.4, 1.78], [1, 2.8]] #n=m-p-1=9-4=5
bs_pts = bsplcurve(p, knot, ctrl, 200)   
fig3 = figbspline2d(ctrl, bs_pts; height=450)
display(fig3)

data: [
  "scatter with fields marker, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, and width"

Let us decrease by 1 the curve degree, as well as the number of knots:

In [9]:
p=2
knot = KnotVector(0, 1, 2, 3, 4, 5, 6, 7) #m=8
ctrl = [[0, 0.0], [-1.0, 2.3], [0.25, 3], [1.4, 1.78], [1, 2.8]] #n=m-p-1=8-3=5
bs_pts = bsplcurve(p, knot, ctrl, 200)   
fig4 = figbspline2d(ctrl, bs_pts; height=450)
display(fig4)

data: [
  "scatter with fields marker, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, and width"

**Closed B-spline curve**

Define the control points and the sequence of knots such that to get a closed B-spline curve, following  *J.J. Risler - Mathematical Methods for CAD*, Cambridge Univ Press.

In [10]:
p = 3
ictrl =  [[0., 90.], [-40, 72], [-18, 20], [-25, -15], [10,-10], [32,20], [25, 60]] #initial control points;
#extend the ictrl to ensure a closed B-spline curve
ctrl = vcat(ictrl, [ictrl[j] for j=1:p]); #by JJ Risler
n = length(ctrl)
u = Float64.(collect(1:n+p+1)) #knots
period = u[n+1]-u[p+1]
u[1:p] = u[n-p+1:n] .- period
u[n+2:n+p+1] = u[p+2:2*p+1] .+ period
knot = KnotVector(u)
bs_pts = bsplcurve(p, knot, ctrl, 200) 
fig5 = figbspline2d(ctrl, bs_pts; height=450)
relayout!(fig5, title_text="Closed B-spline curve", title_x=0.45)
display(fig5)

data: [
  "scatter with fields marker, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, title, and width"

 **Fitting a B-spline to 2D data**

In [11]:
p = 3
knot = KnotVector(range(-2π,2π,length=8))+p*KnotVector(-2π,2π)
f(x) = [x, sin(0.75*sqrt(2)*x)]
BS = BSplineSpace{p}(knot)
ctrl = fittingcontrolpoints(f, (BS,))
bs_pts = bsplcurve(p, knot, ctrl, 200)   
fig6 = figbspline2d(ctrl, bs_pts; width=700, height=350)
display(fig6)

data: [
  "scatter with fields marker, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, and width"

**Fitting a B-spline to 2D data**

In [13]:
p=3
knot = p*KnotVector(0, 6π)+ KnotVector(range(0, 6π, length=12))


KnotVector([0.0, 0.0, 0.0, 0.0, 1.7135959928671598, 3.4271919857343196, 5.140787978601479, 6.854383971468639, 8.5679799643358, 10.281575957202959, 11.99517195007012, 13.708767942937278, 15.422363935804439, 17.1359599286716, 18.84955592153876, 18.84955592153876, 18.84955592153876, 18.84955592153876])

In [14]:
length(knot)

18

In [15]:
f(t) = [2*cos(t), 2*sin(t), 1.5*t]
BS = BSplineSpace{p}(knot)
ctrl = fittingcontrolpoints(f, (BS,))
bs_pts = bsplcurve(p, knot, ctrl, 200); 

In [16]:
c = mapreduce(permutedims, vcat, ctrl)
bp = mapreduce(permutedims, vcat, bs_pts)
   
fig7 = Plot(scatter3d(x=c[:, 1], y=c[:, 2],  z=c[:, 3],        
                       name="ctrl polygon", marker_size=5),
               Layout(width=600, height=600, scene_camera_eye=attr(x=1.65, y=1.65, z=1)))
addtraces!(fig7, scatter3d(x=bp[:,1], y=bp[:, 2], z=bp[:, 3],    
                            name="b-spline", mode="lines", line_color="red"))
display(fig7)    

data: [
  "scatter3d with fields marker, name, type, x, y, and z",
  "scatter3d with fields line, mode, name, type, x, y, and z"
]

layout: "layout with fields height, margin, scene, template, and width"